In [ ]:
import pandas as pd


In [ ]:
# Carregar o arquivo Excel
sheet_data = pd.read_excel(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Pendencias\Arquivos\Mapa2.xlsx", sheet_name='Sheet1')

# Agora o sheet_data contém as novas colunas 'Pendências Total', 'Estoque Total' e 'Saídas Total'
sheet_data.columns

In [ ]:
sheet_data['pendencia_total'] = sheet_data[['CE04_zva70', 'CE05_zva70', 'CE11_zva70', 'CE13_zva70']].sum(axis=1)
sheet_data['estoque_total'] = sheet_data[['CE01_zstok', 'CE03_zstok', 'CE05_zstok', 'CE07_zstok', 'CE09_zstok', 'CE11_zstok', 'CE13_zstok']].sum(axis=1)
sheet_data['FUP_total'] = sheet_data[['Nov/24_FUP', 'Dez/24_FUP', 'Jan/24_FUP', 'Fev/24_Fup']].sum(axis=1)
sheet_data['estoque_no_similar'] = sheet_data['Quant estoque 09/2024'] - sheet_data['estoque_total']

In [ ]:
sheet_data

In [ ]:
sheet_data['projecao_compra'] = sheet_data['pendencia_total'] - (sheet_data['estoque_total'] + sheet_data['FUP_total'] + sheet_data['estoque_no_similar'])
sheet_data['projecao_compra'] = sheet_data['projecao_compra'].apply(lambda x: max(x, 0))



In [ ]:
sheet_data

In [ ]:

caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Mapa Analisado1.0.xlsx'

# Exportar o DataFrame para o arquivo Excel
sheet_data.to_excel(caminho_saida, index=True)

In [ ]:
def atender_estoque_ajustado_v2(row):
    if row['projecao_compra'] == 0:
        # Verificando a ordem de prioridade dos estoques e se o valor é suficiente para cobrir a pendência
        for col in ['CE05_zstok', 'CE13_zstok', 'CE11_zstok', 'CE09_zstok', 'CE07_zstok', 'CE01_zstok', 'estoque_total', 'estoque_no_similar']:
            if row[col] >= row['pendencia_total']:
                if col == 'estoque_no_similar':
                    return "Atender Pelo Similar"
                else:
                    return f"Atender pelo {col.split('_')[0]}"
        # Se todos os estoques forem insuficientes, verifica o FUP total
        if row['FUP_total'] >= row['pendencia_total']:
            return "Aguardar IMP"
        else:
            return "Sem estoque disponível"
    else:
        return row['projecao_compra']  # Retorna o valor da projeção de compra se houver necessidade

# Aplicando a função ajustada ao DataFrame
sheet_data['priorizacao_estoque'] = sheet_data.apply(atender_estoque_ajustado_v2, axis=1)



In [ ]:
sheet_data

In [ ]:

caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Arquivos\Mapa Analisado2.0.xlsx'

# Exportar o DataFrame para o arquivo Excel
sheet_data.to_excel(caminho_saida, index=True)